In [1]:
from DSSATTools import (
    crop, # module that contains the classes for the different crops.
    WeatherStation, # class needed to define the Weather. 
    SoilProfile, SoilLayer, # classes that define the entire soil profile, and the soil layers within the soil profile
    filex, # module that contains the clases for the FileX sections.
    DSSAT # the simulation environment class.
)
import pandas as pd
from datetime import datetime
import tempfile
import os

d:\Final-Year-Project\.venv\Lib\site-packages\DSSATTools\__init__.py:105: DeprecationWarning: DSSATTools version 3.0.0 is a major upgrade and will not be backwards compatible with previous versions. If you are running code that was developed  using a previous DSSATTools version, then install DSSATTools version 2.1.6
  warnings.warn(


In [2]:
PLANTING_DATE = datetime(2024,1,12)

In [3]:
weather = WeatherStation.from_files([r"D:\Final-Year-Project\env_trails\\weather\CHEN2401.WTH"])
weather

WeatherStation(insi='CHEN', lat=12.381, long=78.937, elev=6.0, tav=27.3, amp=11.2, refht=2.0, wndht=2.0, cco2=421.0, table=
   DATE  SRAD  TMAX  TMIN  RAIN  DEWP  WIND   PAR  EVAP  RHUM 
2024001  18.2  29.7  16.3   0.0  16.9 171.1  17.0   1.0  76.0
2024002  14.1  28.6  17.0   0.0  17.2 192.7  17.0   0.8  77.3
2024003  13.8  29.1  17.9   0.1  18.4 203.0  17.0   0.7  79.2
2024004  11.8  28.0  19.0   0.1  18.9 184.9  17.0   0.5  80.7
2024005   8.7  30.2  20.2   0.0  18.7 210.0  17.0   0.7  76.3
2024006  11.7  30.1  19.2   0.6  18.1 241.1  17.0   0.6  75.6
...
...)

In [4]:
soil = SoilProfile.from_file(r"SG12387893",r"D:\Final-Year-Project\env_trails\MY_GENERATED_SOIL.SOL")
soil

SoilProfile(name='SG12387893', soil_data_source='', soil_clasification='CL', soil_depth=200.0, soil_series_name='SoilGrids Auto-Generated Profile', site='SoilGrids', country='India', lat=12.381, long=78.937, scs_family='', scom='', salb=0.13, slu1=6.0, sldr=0.5, slro=75.0, slnf=1.0, slpf=1.0, smhb='', smpx='', smke='', table=
  SLB SLMH   SLLL  SDUL  SSAT  SRGF  SSKS  SBDM  SLOC  SLCL  SLSI  SLCF  SLNI  SLHW  SLHB  SCEC  SADC  SLPX  SLPT  SLPO CACO3  SLAL  SLFE  SLMN  SLBS  SLPA  SLPB  SLKE  SLMG  SLNA  SLSU  SLEC  SLCA 
    5 -99   0.233 0.407 0.419 1.000  0.48  1.42  3.13  30.0  32.3  10.4 0.224   7.0   -99  29.2   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99
   15 -99   0.235 0.406 0.418 1.000  0.45  1.43  1.41  30.8  32.3  10.7 0.166   7.1   -99  30.8   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99
   30 -99   0.239 0.400 0.413 1.000  0.37  1.47  1.23  33.1  30.4  11.0 0.103

In [5]:
cultivator = crop.Maize("IB0024")
cultivator

CropPars(vrname='PIO 3720', expno='.', eco#=CropPars(econame='GENERIC MIDWEST1', tbase=8.0, topt=34.0, ropt=34.0, p20=12.5, djti=4.0, gdde=6.0, dsgft=170.0, rue=4.2, kcan=0.85, tsen=nan, cday=nan), p1=180.0, p2=0.8, p5=685.0, g2=907.9, g3=10.0, phint=38.9)

In [6]:
field = filex.Field(
    id_field='ITHY0001', wsta=weather, flob=0, fldt='DR000', 
    fldd=0, flds=0, id_soil=soil
)


# --- INITIAL CONDITIONS (Based on your MY_GENERATED_SOIL.SOL file) ---
# This block provides a complete, stable set of initial conditions for the simulation.
initial_conditions = filex.InitialConditions(
    pcr='SG',         # Previous Crop is Sorghum
    icdat=PLANTING_DATE, # Sets the date to the planting date
    icrt=500,         # Initial root weight (kg/ha)
    icnd=0,           # Initial nodule weight (not used for Sorghum)
    icrn=1,           # Initial root N concentration
    icre=1,           # Initial residue N concentration
    icres=1300,       # Initial residue amount (kg/ha)
    icren=0.5,        # N % in residue
    icrep=0,          # % incorporation of residue
    icrip=100,        # % incorporation of P
    icrid=10,         # Depth of incorporation (cm)
    table=pd.DataFrame([
        # (Layer Depth, Soil Water, NH4, NO3) - values based on your SOIL.SOL file
        (5,   0.35, 1.5, 1.5),
        (15,  0.35, 1.5, 1.5),
        (30,  0.35, 1.5, 1.5),
        (60,  0.35, 1.0, 1.0),
        (100, 0.35, 1.0, 1.0),
        (200, 0.35, 1.0, 1.0),
    ], columns=['icbl', 'sh2o', 'snh4', 'sno3'])
)
# --- END OF SECTION ---

In [7]:
# Define planting
planting = filex.Planting(
    pdate=PLANTING_DATE, ppop=18, ppoe=18, plme='S',
    plds='R', plrs=45, plrd=0, pldp=5
)

In [8]:
simulation_controls = filex.SimulationControls(
    general=filex.SCGeneral(sdate=PLANTING_DATE),
    options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
    methods=filex.SCMethods(infil='S'),
    management=filex.SCManagement(irrig='N', ferti='R', resid='N', harvs='M')
)

In [9]:
dssat  = DSSAT("./dummy/")

./dummy/ created.


In [10]:
result = dssat.run_treatment(
    field=field,
    cultivar=cultivator,
    planting=planting,
    simulation_controls=simulation_controls
)


RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 MZ   1 -99 -99     0     0     1     0     0   314     0     0  24494  187


OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 24012, at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.